In [1]:
import requests      
import numpy as np   
import pandas as pd 
from bs4 import BeautifulSoup
import calendar

# Spaceweatherlive parser 
Parse https://www.spaceweatherlive.com/en/archive/ for creating database of Solar ARs with coordinates

In [57]:
def get_html(link):
    response = requests.get(link)
    #if response.status_code != 200:
    print(f'For {link} response is {response}')
    return response.content

In [3]:
def parse_sunspot(html):
    soup = BeautifulSoup(html,'html.parser')
    obj = soup.findAll(id, attrs = {'class':'table table-condensed table-bordered'})
    result = {}
    for region in obj:
        region_id = region.thead.tr.th.h3.text.split()[1]
        data = region.tbody.tr.text.strip().split('\n')
        data.remove('')
        data[0] = data[0].split()[0]
        data[1] = data[1].split()[0]
        mag_class = region.tbody.tr.find(attrs = {'class':'region_mag'}).attrs['class'][1]
        data.append(mag_class)
        result[region_id] = data
    return result 

In [59]:
year = [2017] #range(1996, 2017)
month = range(1, 13)

dates = [(y, m, d) for y in year for m in month for d in calendar.Calendar().itermonthdays(y, m) if d != 0]

link_template = 'https://www.spaceweatherlive.com/en/archive/{}/{}/{}/dayobs'
links = [link_template.format(*date) for date in dates]

In [60]:
data = {}
for i, page_link in enumerate(links):
    html = get_html(page_link)
    result = parse_sunspot(html)
    if result:
        data['{}-{}-{}'.format(*dates[i])] = parse_sunspot(html)   

For https://www.spaceweatherlive.com/en/archive/1996/1/1/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/2/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/3/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/4/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/5/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/6/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/7/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/8/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/9/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/10/dayobs response is <Response [200]>
For https://www.spaceweatherlive.com/en/archive/1996/1/11/dayobs resp

In [107]:
def df_from_dict(d):
    dict_of_df = {k: pd.DataFrame(v) for k,v in d.items()}
    df = pd.concat(dict_of_df, axis=1).transpose()
    df.index.names = ['date', 'region']
    df.columns = ['number_of_ss', 'size', 'class', 'location', 'magn_class']
    df.index = df.index.set_levels([pd.to_datetime(df.index.levels[0]), df.index.levels[1].astype(int)])
    df = df.sort_index()
    return df

In [108]:
spot = df_from_dict(data)

In [113]:
spot.query('region == 7969')

,,number_of_ss,size,class,location,magn_class
date,region,,,,,
1996-06-05,7969,1,0,AXX,N09W33,A
1996-06-06,7969,4,10,BXO,N08W44,B


In [13]:
spot.to_pickle('sunspot_1996_2017.pkl')

NameError: name 'spot' is not defined

In [1]:
import pickle
import pandas as pd

In [2]:
sunspots = pd.read_pickle('sunspot_1996_2017.pkl')

In [4]:
sunspots.tail()

,,number_of_ss,size,class,location,magn_class
date,region,,,,,
2017-12-24,12692,12,90,DAO,N18E03,B
2017-12-25,12692,12,160,EAI,N18W09,B
2017-12-26,12692,7,100,EAO,N18W23,B
2017-12-27,12692,4,30,CRO,N18W35,B
2017-12-28,12692,1,10,AXX,N17W46,A


In [4]:
sunspots.index.names

FrozenList(['date', 'region'])

In [62]:
print(sunspots.loc['2010-10-02':'2010-10-03'])
sunspots.loc[['2010-10-02','11109']]

                  number_of_ss size class location magn_class
date       region                                            
2010-10-02 11109            11  270   DHO   N21W51          B
           11110             1    0   AXX   N20W71          A
           11111             2   10   BXO   N20E30          B
2010-10-03 11109             8  320   CKO   N21W65          B
           11110             1    0   AXX   N21W84          A
           11111             3   30   CRO   N24E21          B


number_of_ss size class location magn_class
date       region                                            
2010-10-02 11109            11  270   DHO   N21W51          B
           11110             1    0   AXX   N20W71          A
           11111             2   10   BXO   N20E30          B

In [76]:
sunspots.loc[('2010-10-02',11109)]

number_of_ss        11
size               270
class              DHO
location        N21W51
magn_class           B
Name: (2010-10-02 00:00:00, 11109), dtype: object

In [15]:
sunspots.xs(12494, level='region')

,number_of_ss,size,class,location,magn_class
date,,,,,
2016-02-05,7,150,DAI,S12E09,BGD
2016-02-06,16,230,DAI,S12W04,BG
2016-02-07,9,270,DAI,S11W17,B
2016-02-08,9,270,DKI,S12W29,BD
2016-02-09,8,240,DAI,S12W43,BG
2016-02-10,6,120,DAO,S12W57,B
2016-02-11,5,70,DSO,S12W70,B
2016-02-12,3,40,DSO,S12W84,B
2016-02-13,1,10,AXX,S14W96,A


In [10]:
sunspotsHMI = sunspots.loc['2010-05-01':'2017-12-28']
sunspotsHMI.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11929 entries, (2010-05-01 00:00:00, 11064) to (2017-12-28 00:00:00, 12692)
Data columns (total 5 columns):
number_of_ss    11929 non-null object
size            11929 non-null object
class           11929 non-null object
location        11929 non-null object
magn_class      11929 non-null object
dtypes: object(5)
memory usage: 602.2+ KB


In [11]:
sunspotsHMI.loc[sunspostHMI['magn_class']=='BGD']

,,number_of_ss,size,class,location,magn_class
date,region,,,,,
2011-02-16,11158,36,600,EKC,S21W27,BGD
2011-02-17,11158,19,620,EKC,S21W39,BGD
2011-02-18,11158,17,290,EAC,S19W52,BGD
2011-02-19,11158,25,310,EHC,S19W64,BGD
2011-02-20,11161,19,250,EKC,N11W15,BGD
2011-03-02,11164,31,550,EKC,N25E20,BGD
2011-03-03,11164,41,400,EKC,N25E08,BGD
2011-03-05,11164,27,570,EKC,N25W19,BGD
2011-03-06,11164,28,770,FKC,N24W34,BGD


In [12]:
sunspotsHMI['class'].value_counts()

HSX    1773
BXO    1294
AXX    1114
CSO     909
CAO     828
DAO     785
CRO     692
DSO     606
HAX     555
DAI     503
HRX     338
EKC     286
DRO     158
DSI     150
DAC     144
FKC     142
EAI     128
DKC     121
EAC     103
ESO     101
EKI     100
EAO      88
CAI      86
DKO      76
DKI      73
ESI      71
HHX      62
EKO      55
DHO      55
CKO      53
HKX      46
CHO      43
EHI      36
EHO      35
CSI      33
CRI      29
FKO      26
DHI      24
DSC      20
DRI      20
EHC      18
FKI      16
DHC      15
FSO      15
FHC      13
FAI      13
BXI      12
ESC      11
FHO       9
FAC       9
FSI       7
FAO       7
CKI       6
FHI       5
ERO       4
CHI       3
FSC       3
ERI       1
HSO       1
Name: class, dtype: int64

In [23]:
sunspotsH = sunspots.loc[sunspots['class'].str.contains("E"),:]

In [25]:
sunspotsH['class']

date        region
1996-07-10  7978      EAC
1996-07-11  7978      EHI
1996-07-12  7978      EHO
1996-07-13  7978      EHO
1996-07-30  7981      ESI
1996-07-31  7981      EKI
1996-08-01  7981      ESO
1996-08-02  7981      ESI
1996-08-03  7981      ESO
1996-08-04  7981      ESO
1996-11-26  7999      EKC
1996-11-27  7999      EKI
1996-11-28  7999      EKI
1996-11-29  7999      EKI
1996-11-30  7999      EKI
1996-12-01  7999      EKI
1996-12-02  7999      EKI
1996-12-16  8003      ESO
1997-03-31  8026      ESO
1997-04-03  8026      EAI
1997-04-04  8026      EAO
1997-04-16  8031      EAO
1997-05-24  8040      ESO
1997-08-30  8076      ESI
1997-08-31  8076      ESI
1997-09-01  8076      ESI
1997-09-04  8076      ESI
1997-09-05  8076      ESO
1997-09-08  8085      EAO
1997-09-12  8084      EAI
                     ... 
2017-04-03  12645     EHC
2017-04-04  12645     EKC
2017-04-05  12644     EAO
            12645     EKC
2017-04-06  12644     EAO
2017-04-07  12645     EKC
2017-06-04  12661  